In [1]:
"""
Author 1: Sai Mahesh
Roll No: CS18BTECH11001

Author 2: Vamshi Krishna
Roll No: CS18BTECH11024

Author 3: Sai Varshittha
Roll No: CS18BTECH11035
"""

'\nAuthor 1: Sai Mahesh\nRoll No: CS18BTECH11001\n\nAuthor 2: Vamshi Krishna\nRoll No: CS18BTECH11024\n\nAuthor 3: Sai Varshittha\nRoll No: CS18BTECH11035\n'

In [2]:
# Modules to be used
import numpy as np
import pandas as pd
from collections import defaultdict
import pathlib

In [3]:
# Dataset to be used for importing stock flow graph
PATH = pathlib.Path('./sample_graph.csv')

In [4]:
# function to scan array from input
def scan_array_from_input(num):
    arr = []
    for i in range(num):
        arr.append(int(input()))
    return arr

# function to process input
def process_input(path: pathlib.Path):
    # Input good nodes
    # no_good_nodes = 4
    no_good_nodes = int(input("Number of good Nodes: "))
    print("Enter good nodes: ")
    good_nodes = scan_array_from_input(no_good_nodes)
    # good_nodes = [0, 1, 2, 3]

    # Input bad nodes
    # no_bad_nodes = 3
    no_bad_nodes = int(input("Number of bad Nodes: "))
    print("Enter bad nodes: ")
    bad_nodes = scan_array_from_input(no_bad_nodes)
    # bad_nodes = [4, 5, 6]

    # Input alpha
    # alpha = float(input("Enter Damping factor: "))
    alpha = 0.85

    # Input graph from dataset
    df = pd.read_csv(path)
    nodes_set = []
    outlinks = defaultdict(list)
    inlinks = defaultdict(list)
    maxer = 0

    for index, row in df.iterrows():
        p, q = row['v1'], row['v2']
        maxer = max(maxer, p, q)
        outlinks[p].append(q)
        inlinks[q].append(p)
        nodes_set.append(p)
        nodes_set.append(q)

    nodes_set = list(set(nodes_set))
    maxer += 1

    # print(maxer, nodes_set, outlinks, inlinks)
    return good_nodes, bad_nodes, alpha, maxer, nodes_set, outlinks, inlinks 

In [5]:
# Class for trust rank
class TrustRank:
    # Constructor
    def __init__(self, good_nodes, bad_nodes, alpha, maxer, nodes, outlinks, inlinks, max_iterations, L):
        self.max_iterations = max_iterations
        self.maxer = maxer
        self.good_nodes = good_nodes
        self.alpha = alpha
        self.bad_nodes = bad_nodes
        self.outlinks = outlinks
        self.nodes = nodes
        self.L = L
        self.inlinks = inlinks
    
    # Helper function to calculate matrix
    def calculate_matrix(self, links):
        mat = np.zeros((self.maxer, self.maxer))
        for node, nodes_list in links.items():
            frac = 1/len(nodes_list)
            for node_list_node in nodes_list:
                mat[node_list_node][node] = frac
        return mat

    # function to calculate trust rank scores
    def compute_trust_rank_scores(self, dmatrix, T):
        res = dmatrix
        a = res*(1 - self.alpha)
        b = T*self.alpha
        for i in range(max_iterations):
            res = np.dot(b, res) + a
        return res

    # function to calculate inverse page rank
    def inversePageRank(self, U, dmatrix):
        s = self.compute_trust_rank_scores(dmatrix, U)

        page_rank_score = {i: s[i] for i in range(len(s))}
        page_rank_score_sorted = {k: v for k,v in sorted(page_rank_score.items(), key=lambda item: item[1], reverse=True)}
        return page_rank_score_sorted

    # function to calculate trusted pages
    def getTrustedPages(self, inverse_pagerank_scores):
        trusted_pages = []
        for i, j in inverse_pagerank_scores.items():
            if len(trusted_pages) >= self.L:
                break
            else:
                trusted_pages.append(i)
        return trusted_pages

    # Trust Rank algorithm
    def trustRank(self):
        # Transition Matrix
        T = self.calculate_matrix(self.outlinks)

        # Inverse Transition Matrix
        U = self.calculate_matrix(self.inlinks)

        # Static Score Distribution
        dmatrix = np.zeros(self.maxer)
        for i in self.nodes:
            dmatrix[i] = 1/len(self.nodes)
        
        # Inverse PageRank Scores
        inverse_pagerank_scores = self.inversePageRank(U, dmatrix)

        # Fetch Trusted Pages
        trusted_pages = self.getTrustedPages(inverse_pagerank_scores)

        # Static Score Distribution
        dmatrix = np.zeros(self.maxer)
        score_length = 0
        for i in range(1, self.maxer):
            if i in self.good_nodes:
                if i in trusted_pages:
                    score_length += 1
                    dmatrix[i] = 1
        
        # Normalising dmatriz
        if score_length != 0:
            for i in range(1, self.maxer):
                if dmatrix[i] == 1:
                    dmatrix[i] = 1/score_length

        res = self.compute_trust_rank_scores(dmatrix, T)
        return res

In [6]:
# Get inputs
good_nodes, bad_nodes, alpha, maxer, nodes, outlinks, inlinks = process_input(PATH)
max_iterations = 20
L = 3
trust_rank = TrustRank(good_nodes, bad_nodes, alpha, maxer, nodes, outlinks, inlinks, max_iterations, L)
trust_scores = trust_rank.trustRank()
print("Trust Scores of nodes: ")
print(trust_scores)

Number of good Nodes: 4
Enter good nodes: 
0
1
2
3
Number of bad Nodes: 3
Enter bad nodes: 
4
5
6
Trust Scores of nodes: 
[0.         0.17977109 0.12307085 0.15139467 0.1288946  0.0547239
 0.0547239 ]


# Readme:
for sample input you can enter
1. number of good nodes = 4
2. good nodes = [0, 1, 2, 3]
3. number of bad nodes = 3
4. bad_nodes = [4, 5, 6]